# [Generate synthetic and simulated data for evaluation](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/simulator-interaction-data)
**Azure AI Evaluation SDK's** `Simulator` provides an end-to-end synthetic data generation capability to help developers test their application's response to typical user queries in the absence of production data. AI developers can use an index or text-based query generator and fully customizable simulator to create robust test datasets around non-adversarial tasks specific to their application. The `Simulator` class is a powerful tool designed to generate synthetic conversations and simulate task-based interactions. This capability is useful for:
- **Testing Conversational Applications**: Ensure your chatbots and virtual assistants respond accurately under various scenarios.
- **Training AI Models**: Generate diverse datasets to train and fine-tune machine learning models.
- **Generating Datasets**: Create extensive conversation logs for analysis and development purposes.
By automating the creation of synthetic data, the Simulator class helps streamline the development and testing processes, ensuring your applications are robust and reliable.
<br/>
By automating the creation of synthetic data, the `Simulator` class helps streamline the development and testing processes, ensuring your applications are robust and reliable.

In [1]:
# !az login

In [2]:
# Constants and Libraries
import os, json
from azure.identity import DefaultAzureCredential, get_bearer_token_provider #requires azure-identity
from pprint import pprint
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ["OPENAI_API_VERSION"]

credential = DefaultAzureCredential()

In [3]:
# Initialize Azure OpenAI connection

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("MODEL_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

## Generate text or index-based synthetic data as input
In the first part, we prepare the text for generating the input to our simulator

In [4]:
import asyncio
from azure.identity import DefaultAzureCredential
import wikipedia
import os
from typing import List, Dict, Any, Optional
# Prepare the text to send to the simulator
wiki_search_term = "Leonardo da Vinci"
wiki_title = wikipedia.search(wiki_search_term)[0]
wiki_page = wikipedia.page(wiki_title)
text = wiki_page.summary[:5000]
print(f"{text[:100]}...")

Leonardo di ser Piero da Vinci (15 April 1452 – 2 May 1519) was an Italian polymath of the High Rena...


## Specify application Prompty
The following `application.prompty` file specifies how a chat application behaves.

In [5]:
%%writefile ./eval_assets/application.prompty
---
name: ApplicationPrompty
description: Chat RAG application
model:
    api: chat
    parameters:
        temperature: 0.0
        top_p: 1.0
        presence_penalty: 0
        frequency_penalty: 0
        response_format:
            type: text
 
inputs:
    context:
        type: string
    query:
        type: string
    conversation_history:
        type: dict
---
system:
You are a helpful assistant and you're helping with the user's query. 
Keep the conversation engaging and interesting.

Keep your conversation grounded in the provided context: 
{{ context }}

Output with a string that continues the conversation, responding to 
the latest message from the user query using it same language:
{{ query }}

given the conversation history:
{{ conversation_history }}

Overwriting ./eval_assets/application.prompty


In [6]:
from promptflow.client import load_flow
from pprint import pprint

model_config["type"] = "AzureOpenAI"

prompty_path = "eval_assets/application.prompty"
flow = load_flow(source=prompty_path, model={"configuration": model_config})

pprint(flow(context="", query="Come preparare una pizza fatta in casa", conversation_history=[]))

('Certo! Ecco una ricetta semplice per preparare una pizza fatta in casa:\n'
 '\n'
 'Ingredienti per l’impasto:\n'
 '- 500 g di farina 00\n'
 '- 300 ml di acqua tiepida\n'
 '- 10 g di sale\n'
 '- 3 g di lievito di birra secco (oppure 10 g di lievito fresco)\n'
 '- 1 cucchiaio di olio extravergine d’oliva\n'
 '- 1 cucchiaino di zucchero\n'
 '\n'
 'Procedimento:\n'
 '1. Sciogli il lievito e lo zucchero nell’acqua tiepida e lascia riposare per '
 '10 minuti.\n'
 '2. In una ciotola capiente, versa la farina e aggiungi il sale. Fai una '
 'fontana al centro e versa l’acqua con il lievito e l’olio.\n'
 '3. Impasta fino a ottenere un composto liscio ed elastico. Copri con un '
 'canovaccio e lascia lievitare per almeno 2 ore, fino al raddoppio del '
 'volume.\n'
 '4. Stendi l’impasto su una teglia leggermente unta o su carta forno.\n'
 '5. Condisci con salsa di pomodoro, mozzarella e gli ingredienti che '
 'preferisci (prosciutto, funghi, verdure, ecc.).\n'
 '6. Cuoci in forno preriscaldato a

In [7]:
context = "Conversazione amichevole fra due persone"

conversation_history = [
    {
        "content": "Cosa fai di bello questa sera?",
        "role": "user",
        "context": "Domanda amichevole",
    },
    {
        "content": "Vado al cinema",
        "role": "assistant",
        "context": "Esposizione di un'attività ricreativa divertente",
    },
    {
        "content": "Ah fantastico! Che cosa vai a vedere?",
        "role": "user",
        "context": "Ulteriore domanda amichevole a dimostrare interesse verso l'interlocutore",
    },
    {
        "content": "Lo SQUALO di Steven Spielberg",
        "role": "assistant",
        "context": "Informazione specifica di un'attività divertente che si intende compiere",
    },
]

flow(context=conversation_history[-1]["context"], query=conversation_history[-1]["content"], conversation_history=conversation_history)

'Che bello! È un grande classico, nonostante gli anni fa ancora paura! Lo rivedi per nostalgia o è la prima volta che lo guardi al cinema?'

In [8]:
i = 0
for ch in conversation_history:
    print(f'Message {i} - {ch["role"]}: {ch["content"]}\n')
    i += 1
    
response = flow(context=conversation_history[-1]["context"], query=conversation_history[-1]["content"], conversation_history=conversation_history)

conversation_history.append({"content": response, "role": "user" if conversation_history[-1]["role"]=="assistant" else "assistant", "context": ""})
print(f'Message {i} - {conversation_history[-1]["role"]}: {conversation_history[-1]["content"]}\n')

Message 0 - user: Cosa fai di bello questa sera?

Message 1 - assistant: Vado al cinema

Message 2 - user: Ah fantastico! Che cosa vai a vedere?

Message 3 - assistant: Lo SQUALO di Steven Spielberg

Message 4 - user: Che bello! Un grande classico, non passa mai di moda. È la prima volta che lo vedi al cinema o lo hai già visto?



## Specify target callback to simulate against
You can bring any application endpoint to simulate against by specifying a target callback function such as the following given an application that is an LLM with a Prompty file like `application.prompty`

In [9]:
async def callback(
    messages: Dict,
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
    subfolder: str = "eval_assets",
) -> dict:
    messages_list = messages["messages"]
    # Get the last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = latest_message.get("context", None) # looks for context, default None
    # Call your endpoint or AI application here
    prompty_path = os.path.join(os.getcwd(), subfolder, "application.prompty") 
    _flow = load_flow(source=prompty_path, model={"configuration": model_config})
    response = _flow(query=query, context=context, conversation_history=messages_list)
    # Format the response to follow the OpenAI chat protocol
    formatted_response = {
        "content": response,
        "role": "user" if conversation_history[-1]["role"]=="assistant" else "assistant",
        "context": context,
    }
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context
    }

In [10]:
from azure.ai.evaluation.simulator import Simulator
# model_config["type"] = "AzureOpenAI" # "azure_openai" or "AzureOpenAI" 
simulator = Simulator(model_config=model_config)

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [11]:
outputs = await simulator(
    target=callback,
    text="Il Duomo di Milano è una delle cattedrali più iconiche al mondo e il simbolo della città. La sua costruzione iniziò nel 1386 e si protrasse per secoli, con il completamento ufficiale nel 1932. È la chiesa più grande d'Italia e una delle più imponenti a livello mondiale",# text,
    num_queries=1,  # Minimal number of queries
)

C:\Users\mauromi\.conda\envs\genai_evaluation\Lib\site-packages\azure\ai\evaluation\simulator\_simulator.py:152: UserWarning: You have specified 'num_queries' > len('tasks') (1 > 0). All tasks will be used for generation and the remaining 1 lines will be simulated in task-free mode
  warnings.warn(
Generating: 100%|████████████████████████████████████████████████| 5/5 [00:37<00:00,  7.58s/message]


In [12]:
outputs

[{'messages': [{'role': 'user',
    'content': 'In which year did the construction of Il Duomo di Milano officially complete?',
    'context': 'None'},
   {'role': 'assistant',
    'content': 'The construction of Il Duomo di Milano (Milan Cathedral) was officially completed in 1965, although the main structure was largely finished by 1813. The cathedral took nearly six centuries to complete!',
    'context': 'None'},
   {'role': 'user',
    'content': "That's fascinating! What architectural styles are represented in the design of Il Duomo di Milano?",
    'context': 'None'},
   {'role': 'assistant',
    'content': 'Il Duomo di Milano is a remarkable blend of architectural styles, primarily showcasing Gothic architecture with its pointed arches, ribbed vaults, and an abundance of spires and statues. However, because its construction spanned several centuries, you can also find elements of Renaissance, Baroque, and even some Neoclassical influences in its design. The intricate façade, fo

In [13]:
i = 0
for ch in outputs[0]["messages"]:
    print(f'Message {i} - {ch["role"]}: {ch["content"]}\n')
    i += 1

Message 0 - user: In which year did the construction of Il Duomo di Milano officially complete?

Message 1 - assistant: The construction of Il Duomo di Milano (Milan Cathedral) was officially completed in 1965, although the main structure was largely finished by 1813. The cathedral took nearly six centuries to complete!

Message 2 - user: That's fascinating! What architectural styles are represented in the design of Il Duomo di Milano?

Message 3 - assistant: Il Duomo di Milano is a remarkable blend of architectural styles, primarily showcasing Gothic architecture with its pointed arches, ribbed vaults, and an abundance of spires and statues. However, because its construction spanned several centuries, you can also find elements of Renaissance, Baroque, and even some Neoclassical influences in its design. The intricate façade, for example, was completed much later and reflects a mix of these styles, making the cathedral a true testament to the evolving tastes and techniques of differen